In [2]:
import re
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as mpatches
from scipy import stats
from scipy.stats import t
import numpy as np
from matplotlib.patches import Patch
from datetime import datetime

In [3]:
models = ['Inception_V3','EfficientNet_B3']
models_features = {'Inception_V3':{'color':['#c6dcff']},'EfficientNet_B3':{'color':['#86cbd6']}}

# Keras models info (Figure 3)

In [87]:
df = pd.read_csv('/data/glusterfs/agurianova/architecture_change/notebooks/keras_model_info.csv', sep = ',')
family_colors = {'Inception':'#c6dcff', 'EfficientNet':'#c2e5eb'}
df['Color'] = df['Family'].map(family_colors)

In [ ]:
fig, ax1 = plt.subplots(figsize=(2.5, 2))

bars = ax1.bar(
    x=df['Model'],
    height=df['Acc@1'],
    color=df['Color'],
    label="Top-1 accuracy",
    edgecolor='black',  # Border color
    linewidth=1 
)
ax1.set_xlabel("Model")
ax1.set_ylabel("Top-1 accuracy (%)", color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.set_xticks(df['Model'])
ax1.set_xticklabels(df['Model'], rotation=90)
#ax1.set_ylim(0,100)

inception_acc = df.loc[df['Model'] == 'Inception_V3', 'Acc@1'].values
ax1.axhline(y=inception_acc, color='black', linestyle='--', linewidth=1, label='Inception_V3')

ax2 = ax1.twinx()
ax2.plot(
    df['Model'],
    df['Params'] ,
    color='black',
    marker='x',
    linestyle='',
    linewidth=1,
    label="Parameters"
)
ax2.set_ylabel("Parameters (Millions)", color='black')
ax2.tick_params(axis='y', labelcolor='black')

# Add Time_GPU line
ax3 = ax1.twinx()
ax3.plot(
    df['Model'],
    df['Time_GPU'],
    color='black',
    marker='o',
    linestyle='-',
    linewidth=1,
    label="GPU Time",
    alpha = 0.5
)
# Position the third y-axis on the right
ax3.spines['right'].set_position(('outward', 40))
ax3.set_ylabel("GPU Time (ms)", color='black')
ax3.tick_params(axis='y', labelcolor='black')

legend_patches = [mpatches.Patch(color=color, label=label) for label, color in family_colors.items() if label in df['Family'].unique()]
ax1.legend(handles=legend_patches, title="Family", bbox_to_anchor=(1.55, 1), loc='upper left')
ax2.legend(loc='upper left',bbox_to_anchor=(1.55, 0.5))
ax3.legend(loc='upper left', bbox_to_anchor=(1.55, 0.27))

plt.title("Performance on ImageNet")
#fig.tight_layout()
plt.show()

In [ ]:
def parse_log(file_path, metric='f1_weighted', part='tune'):
    parsed = []

    model_type = None
    opt = None
    lr = None
    trial = None

    label = None

    timestamp_pattern = r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}'
    first_timestamp = None
    last_timestamp = None

    with open(file_path, 'r') as file:
        for line in file:

            # model
            if model_type is None:
                m = re.search(r'model_type:\s*([^\s]+)', line)
                if m:
                    model_type = m.group(1)

            # values
            matches = re.findall(rf'{part}/{metric}(?:_\d+)?=([\d.e-]+),', line) #re.findall(rf'{part}/{metric}=([\d.e-]+),', line)
            if matches:
                parsed.extend(map(float, matches))

            # optimizer
            if opt is None:
                match = re.search(r'optimizer:\s+([^\s]+)', line) 
                if match:
                    opt = match.group(1)

            # lr
            if lr is None:
                match = re.search(r'learning_rate:\s+([^\s]+)', line)
                if match:
                    lr = float(match.group(1))    

            # trial
            if trial is None:
                match = re.search(r'Optuna. Trial\s+([\d.e-]+)', line)     
                if match:
                    trial = match.group(1)

            # time
            match = re.search(timestamp_pattern, line)
            if match:
                ts_str = match.group(0)
                ts = datetime.strptime(ts_str, '%Y-%m-%d %H:%M:%S,%f')
                if first_timestamp is None:
                    first_timestamp = ts
                last_timestamp = ts     

    present_points = [i for i in range(len(parsed)) if i not in (3, 8)] if part == 'tune' else range(len(parsed)) # skip intermediate validation checks
    parsed = [parsed[i] for i in present_points]          

    parsed_smoothed = []
    weight = 0.5
    last = parsed[0]
    for v in parsed:
        last = last * weight + (1 - weight) * v
        parsed_smoothed.append(last)    

    if metric != 'loss':
        parsed_smoothed = [parsed_smoothed[i] * 100 for i in range(len(parsed_smoothed))]
    
    #label = f"{trial}_{model_type}_{opt}_lr{lr:.1e}"

    if first_timestamp and last_timestamp:
        time_diff = last_timestamp - first_timestamp

    return parsed_smoothed, time_diff


In [11]:
def confidence_interval(curves, confidence_level=0.95):
    n = curves.shape[0]
    mean_curve = np.mean(curves, axis=0)
    std_curve = np.std(curves, axis=0, ddof=1)
    stderr = std_curve / np.sqrt(n)
    
    t_crit = t.ppf((1 + confidence_level) / 2, df=n - 1)
    
    margin_of_error = t_crit * stderr
    
    lower_bound = mean_curve - margin_of_error
    upper_bound = mean_curve + margin_of_error
    
    return lower_bound, upper_bound

# results

In [1]:
!pwd

/data/glusterfs/agurianova/architecture_change/notebooks


In [ ]:
result_paths = {'Inception_V3':{'training':[f"./inception/optuna_trial_{i}/logs.log" for i in range(10)], 
                                'testing':[f'./inception/optuna_trial_{i}/test/happy.output.summary.csv' for i in range(10)]},
                'EfficientNet_B3':{'training':[f"./efficientnetb3/optuna_trial_{i}/logs.log" for i in range(10)], 
                                    'testing':[f'./efficientnetb3/optuna_trial_{i}/test/happy.output.summary.csv' for i in range(10)]}}

## training

In [8]:
metrics = ['f1','precision','recall']
metrics_features = {'f1':{'pattern':'xxxx','name':'f1_weighted'},
                    'precision':{'pattern':r"\\\\",'name':'precision'},
                    'recall':{'pattern':'////','name':'recall'}}

parts_features = {'train':{'name':'training', 'steps_per_epoch':2778 / 5}, #(all steps / logging step)
                  'tune':{'name':'validation','steps_per_epoch':1}}

### training-validation: loss

In [ ]:
fig, axes = plt.subplots(
    nrows = 1, ncols = 2,
    figsize = (6,3),
    sharex = True, sharey = False,
    constrained_layout = True
)

configurations = [{'metric_key': 'loss', 'part': 'train'},
                  {'metric_key': 'loss', 'part': 'tune'}]

legend_handles = []
legend_labels = []

for ax, cfg in zip(axes, configurations):

    for i, model in enumerate(models):

        paths = result_paths[model]['training']
        curves = []
        for path in paths:
            curve, label = parse_log(path, cfg['metric_key'], cfg['part'])
            #curves.append(smooth_curve(curve,weight=0.4))
            curves.append(curve)

        curves = np.array(curves)
        mean_curve = np.mean(curves, axis=0)
        lower_bound, upper_bound = confidence_interval(curves, 0.95)

        epochs = np.arange(1, curves.shape[1] + 1)

        line, = ax.plot(epochs / parts_features[cfg['part']]['steps_per_epoch'], 
                mean_curve, 
                color=models_features[model]['color'][0], 
                linewidth=2)
        ax.fill_between(epochs / parts_features[cfg['part']]['steps_per_epoch'], 
                        lower_bound, upper_bound, 
                        color=models_features[model]['color'][0], 
                        alpha=0.2)
        
        if ax == axes[0]:
            legend_handles.append(line)
            legend_labels.append(['Baseline (Inception_V3)', 'Alternative (EfficientNet-B3)'][i])

    ax.set_title(f"{parts_features[cfg['part']]['name'].upper()} {cfg['metric_key'].upper()}", fontsize=13)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel(cfg['metric_key'].upper(), fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_ylim(0.58, 0.66)
    ax.set_xticks(range(0,11,1))
    
fig.legend(
    legend_handles, legend_labels,
    title="Models",
    fontsize=10,
    loc='center left',        
    bbox_to_anchor=(1.02, 0.76))

plt.tight_layout()
plt.show()

### validation: metrics

In [ ]:
fig, axes = plt.subplots(
    nrows = 1, ncols = 3,
    figsize = (8,3),
    sharex = True, sharey = True,
    constrained_layout = True
)

configurations = [{'metric_key': 'f1', 'part': 'tune'},
                  {'metric_key': 'precision', 'part': 'tune'},
                  {'metric_key': 'recall', 'part': 'tune'}]

legend_handles = []
legend_labels = []

for ax, cfg in zip(axes, configurations):

    for i, model in enumerate(models):

        paths = result_paths[model]['training']
        curves = []
        for path in paths:
            curve, label = parse_log(path, metrics_features[cfg['metric_key']]['name'], cfg['part'])
            curves.append(curve)

        curves = np.array(curves)
        mean_curve = np.mean(curves, axis=0)
        lower_bound, upper_bound = confidence_interval(curves, 0.95)

        epochs = np.arange(1, curves.shape[1] + 1)
        line, = ax.plot(epochs / parts_features[cfg['part']]['steps_per_epoch'], 
                mean_curve, 
                color=models_features[model]['color'][0], 
                linewidth=2)
        ax.fill_between(epochs / parts_features[cfg['part']]['steps_per_epoch'], 
                        lower_bound, upper_bound, 
                        color=models_features[model]['color'][0], 
                        alpha=0.2)
        
        if ax == axes[0]:
            legend_handles.append(line)
            legend_labels.append(['Baseline (Inception_V3)', 'Alternative (EfficientNet-B3)'][i])

    ax.set_title(f"{parts_features[cfg['part']]['name'].upper()} {cfg['metric_key'].upper()}", fontsize=13)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel(cfg['metric_key'].upper(), fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_xticks(range(0,11,1))
    
fig.legend(
    legend_handles, legend_labels,
    title="Models",
    fontsize=10,
    loc='center left',        
    bbox_to_anchor=(1.02, 0.76))

plt.tight_layout()
plt.show()

In [ ]:
def confidence_interval_list(data_list, confidence_level=0.95):
    data = np.array(data_list) 
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    stderr = std / np.sqrt(n)
    
    t_crit = t.ppf((1 + confidence_level) / 2, df=n - 1)
    margin_of_error = t_crit * stderr
    
    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error
    
    return lower_bound, upper_bound

In [81]:
tab = {'Model':[],'Metric':[],'Value':[]}
metrics = ['f1_weighted','precision','recall']

for model in models:
    
    for metric in metrics:

        paths = result_paths[model]['training']

        values = []
        for path in paths:
            parsed_values, _ = parse_log(path, f'{metric}', part='tune')
            values.append(parsed_values[-1])

        mean_value = round(np.mean(values), 2)
        ci_l, ci_u = confidence_interval_list(values)
        mean_ci = f'{round(mean_value,2)} (95% CI: {round(ci_l,2)}-{round(ci_u,2)})'

        tab['Model'].append(model)
        tab['Metric'].append(metric)
        tab['Value'].append(mean_ci)

df = pd.DataFrame(tab)

In [ ]:
df.pivot(index='Model', columns='Metric', values='Value').reindex(models)

### validation: metrics per class

In [91]:
classes = ['het','homalt','homref']
classes_features = {'het':{'alleles':'0/1,','pattern':'//'},
                    'homalt':{'alleles':'1/1,','pattern':'xx'},
                    'homref':{'alleles':'0/0,','pattern':''}}

In [92]:
tab = {'Model':[],'Metric':[],'Class':[],'Value':[]}
metrics = ['f1', 'precision', 'recall']

for model in models:
    
    for metric in metrics:
        
        for class_ in classes:
            
            values = []
            for path in result_paths[model]['training']:
                parsed_values, _ = parse_log(path, f'{metric}_{class_}', part='tune')
                values.append(parsed_values[-1])
            mean_value = np.mean(values)

            tab['Model'].append(model)
            tab['Metric'].append(metric)
            tab['Class'].append(class_)
            tab['Value'].append(mean_value)

df = pd.DataFrame(tab)

In [ ]:
custom_index_order = ['homref', 'het', 'homalt']

fig, axes = plt.subplots(
    nrows = 1, ncols = 2,
    figsize = (6,3),
    sharex = True, sharey = False,
    constrained_layout = True
)

configurations = [{'metric_key': 'f1_weighted', 'part': 'tune'},
                  {'metric_key': 'precision', 'part': 'tune'},
                  {'metric_key': 'recall', 'part': 'tune'}]

# axes[0]
pivot_df = df[df['Metric']=='f1'].pivot(index='Class', columns='Model', values='Value').reindex(custom_index_order)
x = np.arange(len(classes))
width = 0.4
for i, model in enumerate(models):

    bar_x_positions = x + i * width - width / 2
    bar_y_values = pivot_df[model]
    bar_colors = models_features[model]['color'] * len(bar_y_values)
    bar_patterns = [metrics_features['f1']['pattern']] * len(bar_y_values)

    bars = axes[0].bar(
        bar_x_positions,
        bar_y_values,
        width,
        color=bar_colors,
        edgecolor='black'
    )

    for bar, pattern in zip(bars, bar_patterns):
        bar.set_hatch(pattern)

axes[0].set_ylabel('Value')
axes[0].set_xlabel('Class')
axes[0].set_title(f"{parts_features['tune']['name'].upper()} {'F1'}", fontsize=13)
axes[0].set_ylim(90,98)
axes[0].set_xticks(x)
axes[0].set_xticklabels([f'{classes_features[i]["alleles"]}{i}' for i in pivot_df.index])

# axes[1]
width = 0.2
side = -(width+0.01)

for i, model in enumerate(models):

    pivot_df = df[(df['Metric']!='f1')&(df['Model']==model)].pivot(index='Class', columns='Metric', values='Value').reindex(custom_index_order) 

    for j, metric in enumerate(['precision','recall']):

        bar_x_positions = (x + j * width - width / 2) + side
        bar_y_values = pivot_df[metric]
        bar_colors = models_features[model]['color'] * len(bar_y_values)
        bar_patterns = [metrics_features[metric]['pattern']] * len(bar_y_values) 

        bars = axes[1].bar(
            bar_x_positions,
            bar_y_values,
            width,
            color=bar_colors,
            edgecolor='black'
        )
        for bar, pattern in zip(bars, bar_patterns):
            bar.set_hatch(pattern)

    side = -side

axes[1].set_ylabel('Value')
axes[1].set_xlabel('Class')
axes[1].set_title(f"{parts_features['tune']['name'].upper()} {'PRECISION, RECALL'}", fontsize=13)
axes[1].set_ylim(90,98)
axes[1].set_xticks(x)
axes[1].set_xticklabels([f'{classes_features[i]["alleles"]}{i}' for i in pivot_df.index])

# legend
legend_handles = [Patch(facecolor=models_features['Inception_V3']['color'][0], hatch="", edgecolor='black', label='Inception_V3'),
                  Patch(facecolor=models_features['EfficientNet_B3']['color'][0], hatch="", edgecolor='black', label='EfficientNet_B3'),
                  Patch(facecolor='white', hatch="xx", edgecolor='black', label='F1'),
                  Patch(facecolor='white', hatch=metrics_features['precision']['pattern'], edgecolor='black', label='PRECISION'),
                  Patch(facecolor='white', hatch=metrics_features['recall']['pattern'], edgecolor='black', label='RECALL')]
legend_labels = ['Baseline (Inception_V3)','Alternative (EfficientNet_B3)','F1','PRECISION','RECALL']
fig.legend(
    legend_handles, legend_labels,
    title="Models and Metrics",
    fontsize=10,
    loc='center left',        
    bbox_to_anchor=(1.03, 0.65))
fig.tight_layout()

## testing

### SNP/INDEL hap.py table

In [97]:
tab = {'Type':[],'Model':[],'Metric':[],'Value':[]}
metrics = ['METRIC.F1_Score','METRIC.Precision','METRIC.Recall']

for type in ['SNP','INDEL']:

    for model in models:
        
        for metric in metrics:

                paths = result_paths[model]['testing']

                values = []
                for path in paths:
                    df = pd.read_csv(path)
                    df = df.loc[(df['Type'] == type) & (df['Filter'] == 'PASS')]
                    value = df[metric].iloc[0]
                    values.append(value)
                values = np.array(values) * 100
                
                mean_value = np.mean(values)
                ci_l, ci_u = confidence_interval_list(values)
                mean_ci = f'{round(mean_value,2)} (95% CI: {round(ci_l,2)}-{round(ci_u,2)})'

                tab['Type'].append(type)
                tab['Model'].append(model)
                tab['Metric'].append(metric)
                tab['Value'].append(mean_ci)

df = pd.DataFrame(tab)

In [ ]:
df[df['Type']=='SNP'].pivot(index='Model', columns='Metric', values='Value').reindex(models)

In [ ]:
df[df['Type']=='INDEL'].pivot(index='Model', columns='Metric', values='Value').reindex(models)

### time

In [ ]:
tab = {'Model':[],'Metric':[],'Value':[]}
metrics = ['f1_weighted','precision','recall']

for model in models:
    
    paths = result_paths[model]['training']

    values = []
    for path in paths:
        parsed_values, time = parse_log(path, f'{metric}', part='tune')
        values.append(time)

    mean_value = np.mean(values)
    
    tab['Model'].append(model)
    tab['Metric'].append(metric)
    tab['Value'].append(mean_value)

df = pd.DataFrame(tab)
df